In [1]:
import gspread
import pandas as pd
from google.oauth2.service_account import Credentials

### Dataset X

In [3]:
SERVICE_ACCOUNT_FILE = 'key/credentials.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly',
          'https://www.googleapis.com/auth/drive']

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
client = gspread.authorize(creds)

sheet = client.open('Recheck HC').worksheet("20250701-0831")
data = sheet.get_all_values()

df = pd.DataFrame(data[1:], columns=data[0])

In [4]:
df_2025 = df.iloc[:, [2,3,17,23,24,25,26]].copy()
df_2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48949 entries, 0 to 48948
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   application_name     48949 non-null  object
 1   call_id              48949 non-null  object
 2   text                 48949 non-null  object
 3   manual_audit_result  48949 non-null  object
 4   checkbox             48949 non-null  object
 5   checker              48949 non-null  object
 6   checking_date        48949 non-null  object
dtypes: object(7)
memory usage: 2.6+ MB


### Dataset Y

In [5]:
df_recheck = pd.read_csv('dataset_qc/recheck_hc.csv')

df_recheck = df_recheck.iloc[:, [2,3,4,8,19,27]]
df_recheck

cols = ['Nama Agent ', 'ASI/AFI', 'Efektif ']

# Date
df_recheck['Tanggal Pengerjaan'] = pd.to_datetime(df_recheck['Tanggal Pengerjaan'])

# Cols Cat
for col in cols:
    df_recheck[col] = df_recheck[col].astype('category')

# count feature clean
df_recheck['Count Efektif'] = df_recheck['Count Efektif'].fillna(0).astype(int)

df_recheck.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31158 entries, 0 to 31157
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Tanggal Pengerjaan  31158 non-null  datetime64[ns]
 1   Nama Agent          31158 non-null  category      
 2   ASI/AFI             31158 non-null  category      
 3   Call ID             31158 non-null  object        
 4   Efektif             31158 non-null  category      
 5   Count Efektif       31158 non-null  int64         
dtypes: category(3), datetime64[ns](1), int64(1), object(1)
memory usage: 822.2+ KB


C:\Users\ozy.prazuganda\AppData\Local\Temp\ipykernel_15724\2325867966.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_recheck = pd.read_csv('dataset_qc/recheck_hc.csv')


### X + Y

In [17]:
data_21 = df_recheck[
    (df_recheck['Tanggal Pengerjaan'] == '2025-11-21')
]

data_21_unique = data_21.drop_duplicates('Call ID ', keep='first').reset_index(drop=True)

data_21_unique['ASI/AFI'].value_counts()

ASI/AFI
ASI     11
AFI      5
Name: count, dtype: int64

In [ ]:
data_21_unique[data_21_unique['ASI/AFI'] == 'AFI']

,Tanggal Pengerjaan,Nama Agent,ASI/AFI,Call ID,Efektif,Count Efektif
2,2025-11-21,Aulia,AFI,7e206329cf714b8a870c2ae728eb9878,Miss Target/ Not HC,0
3,2025-11-21,Aulia,AFI,95ba635125cb46109aa10fc1952e8ec3,Miss Target/ Not HC,0
4,2025-11-21,Aulia,AFI,a9ef1acad6064f04a7667f13e0c10734,Miss Target/ Not HC,0
5,2025-11-21,Aulia,AFI,b9ab26a458174e7f8cf4214c7c39f858,Miss Target/ Not HC,0
11,2025-11-21,Reza,AFI,d74847025a5842c78c4ef63fef59f28e,Miss Target/ Not HC,0


: 